In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import re, string, time, random
import matplotlib.pyplot as plt
from PIL import Image
from kaggle_datasets import KaggleDatasets
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pathlib
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from nltk.corpus import stopwords

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv(r'/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding='latin1')
test_df = pd.read_csv(r'/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='latin1')

In [ ]:
def data_cleansing(corpus):
    corpus = re.sub(r'http\S+', ' ', corpus)
    corpus = re.sub(r'<.*?>',' ', corpus)
    corpus = re.sub(r'\d+',' ', corpus)
    corpus = re.sub(r'#\w+',' ', corpus)
    corpus = re.sub(r'@\w+',' ', corpus)
    corpus = corpus.split()
    corpus = " ".join([word for word in corpus if not word in stop_words])
    return corpus

stop_words = stopwords.words('english')
train_df['OriginalTweet'] = train_df['OriginalTweet'].apply(data_cleansing)

In [ ]:
def dict_cate(df):
    dict_cate = {}
    for i, cate in enumerate(set(df.values)):
        dict_cate[cate] = i
    return dict_cate
dict_cate = dict_cate(train_df['Sentiment'])
dict_cate['Extremely Negative']=0
dict_cate['Neutral']=1
dict_cate['Extremely Positive']=2
dict_cate['Positive']=2
dict_cate['Negative']=0
dict_cate

In [ ]:
train_df['Sentiment'] = train_df['Sentiment'].apply(lambda x: dict_cate[x])

In [ ]:
#TextVectorization
vocab_size=10000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
encoder.adapt(all_df.map(lambda text: text))
def encoder_fn(text, label):
  text = tf.expand_dims(text, -1)
  return encoder(text), label
dataset = dataset.map(encoder_fn)

In [ ]:
#Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_df['OriginalTweet'])
train_df['OriginalTweet'] = tokenizer.texts_to_sequences(train_df['OriginalTweet'])
vocab_size = len(tokenizer.word_index)+1
train_Tweet = pad_sequences(train_df['OriginalTweet'], padding='post')

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=train_Tweet.shape[1]), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True)),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_Tweet, train_df['Sentiment'].values, validation_split=0.1
                    , batch_size=64, epochs=10, shuffle=True)

In [ ]:
plt.plot(np.arange(len(history.history['loss'])), history.history['loss'])
plt.plot(np.arange(len(history.history['loss'])), history.history['val_loss'])

In [ ]:
plt.plot(np.arange(len(history.history['loss'])), history.history['accuracy'])
plt.plot(np.arange(len(history.history['loss'])), history.history['val_accuracy'])

In [ ]:
test_df['OriginalTweet'] = test_df['OriginalTweet'].apply(data_cleansing)
test_df['Sentiment'] = test_df['Sentiment'].apply(lambda x: dict_cate[x])
test_df['OriginalTweet'] = tokenizer.texts_to_sequences(test_df['OriginalTweet'])
test_Tweet = pad_sequences(test_df['OriginalTweet'], padding='post')
test_df

In [ ]:
test_history = model.evaluate(test_Tweet, test_df['Sentiment'])